# SocietyOfMindAgent

This notebook demonstrates the SocietyOfMindAgent, which runs a group chat as an internal monologue, but appears to the external world as a single agent. This confers three distinct advantages:

1. It provides a clean way of producing a hierarchy of agents, hiding complexity as inner monologues.
2. It provides a consistent way of extracting an answer from a lengthy group chat (normally, it is not clear which message is the final response, and the response itself may not always be formatted in a way that makes sense when extracted as a standalone message).
3. It provides a way of recovering when agents exceed their context window constraints (the inner monologue is protected by try-catch blocks)

````{=mdx}
:::info Requirements
Install `pyautogen`:
```bash
pip install pyautogen
```

For more information, please refer to the [installation guide](/docs/installation/).
:::
````

In [1]:
import autogen  # noqa: E402

'''llm_config = {
    "timeout": 600,
    "cache_seed": 44,  # change the seed for different trials
    "config_list": autogen.config_list_from_json(
        "OAI_CONFIG_LIST",
        filter_dict={"model": ["gpt-4", "gpt-4-0613", "gpt-4-32k", "gpt-4-32k-0613", "gpt-4-1106-preview"]},
    ),
    "temperature": 0,
}'''

'llm_config = {\n    "timeout": 600,\n    "cache_seed": 44,  # change the seed for different trials\n    "config_list": autogen.config_list_from_json(\n        "OAI_CONFIG_LIST",\n        filter_dict={"model": ["gpt-4", "gpt-4-0613", "gpt-4-32k", "gpt-4-32k-0613", "gpt-4-1106-preview"]},\n    ),\n    "temperature": 0,\n}'

In [2]:
import os
config_list = [
    {"api_type": "groq", "model": "llama-3.1-70b-versatile", "api_key": os.environ.get("GROQ_API_KEY")},
    {"api_type": "groq", "model": "llama3-70b-8192", "api_key": os.environ.get("GROQ_API_KEY")},
    {"api_type": "groq", "model": "llama3-groq-70b-8192-tool-use-preview", "api_key": os.environ.get("GROQ_API_KEY")},
    {"api_type": "groq", "model": "mixtral-8x7b-32768", "api_key": os.environ.get("GROQ_API_KEY")},
    {"api_type": "groq", "model": "llama3-groq-8b-8192-tool-use-preview", "api_key": os.environ.get("GROQ_API_KEY")},
    {"api_type": "groq", "model": "gemma2-9b-it", "api_key": os.environ.get("GROQ_API_KEY")},
    {"api_type": "groq", "model": "llama-3.1-8b-instant", "api_key": os.environ.get("GROQ_API_KEY")},
    {"api_type": "groq", "model": "llama-guard-3-8b", "api_key": os.environ.get("GROQ_API_KEY")},
    {"api_type": "groq", "model": "llama3-8b-8192", "api_key": os.environ.get("GROQ_API_KEY")},
    ]

llm_config={"config_list" : config_list}

````{=mdx}
:::tip
Learn more about configuring LLMs for agents [here](/docs/topics/llm_configuration).
:::
````

### Example Group Chat with Two Agents

In this example, we will use an AssistantAgent and a UserProxy agent (configured for code execution) to work together to solve a problem. Executing code requires *at least* two conversation turns (one to write the code, and one to execute the code). If the code fails, or needs further refinement, then additional turns may also be needed. We will then wrap these agents in a SocietyOfMindAgent, hiding the internal discussion from other agents (though will still appear in the console), and ensuring that the response is suitable as a standalone message.

#### Construct the Inner-Monologue Agents
We begin by constructing the inner-monologue agents. These are the agents that do that real work.

In [3]:
assistant = autogen.AssistantAgent(
    "inner-assistant",
    llm_config=llm_config,
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
)

code_interpreter = autogen.UserProxyAgent(
    "inner-code-interpreter",
    human_input_mode="NEVER",
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False,
    },
    default_auto_reply="",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
)

groupchat = autogen.GroupChat(
    agents=[assistant, code_interpreter],
    messages=[],
    speaker_selection_method="round_robin",  # With two agents, this is equivalent to a 1:1 conversation.
    allow_repeat_speaker=False,
    max_round=8,
)

manager = autogen.GroupChatManager(
    groupchat=groupchat,
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
)

#### Construct and Run the SocietyOfMind Agent
We now wrap the inner group-chat with the SocietyOfMind Agent, and create a UserProxy to talk to it.

In [4]:
from autogen.agentchat.contrib.society_of_mind_agent import SocietyOfMindAgent  # noqa: E402

task = "On which days in 2024 was Microsoft Stock higher than $370?"

society_of_mind_agent = SocietyOfMindAgent(
    "society_of_mind",
    chat_manager=manager,
    llm_config=llm_config,
)

user_proxy = autogen.UserProxyAgent(
    "user_proxy",
    human_input_mode="NEVER",
    code_execution_config=False,
    default_auto_reply="",
    is_termination_msg=lambda x: True,
)

user_proxy.initiate_chat(society_of_mind_agent, message=task)

user_proxy (to society_of_mind):

On which days in 2024 was Microsoft Stock higher than $370?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
society_of_mind (to chat_manager):

On which days in 2024 was Microsoft Stock higher than $370?

--------------------------------------------------------------------------------

Next speaker: inner-assistant

inner-assistant (to chat_manager):

To find the days in 2024 when Microsoft Stock was higher than $370, we can follow these steps:

1. Collect historical stock data for Microsoft (MSFT) in 2024.
2. Filter the data to find the days when the stock price was higher than $370.

Since we need to collect historical stock data, I'll suggest Python code that uses the yfinance library to fetch the data and prints the required days.

```python
# filename: msft_stock.py
import yfinance as yf
from datetime import datetime

# Download historical stock data for Microsoft (MSFT) from start of

/home/tjamil/.local/share/virtualenvs/Agents-4yRd6dJo/lib/python3.11/site-packages/autogen/oai/groq.py:286: UserWarning: Cost calculation not available for model llama-3.1-70b-versatile
  warnings.warn(f"Cost calculation not available for model {model}", UserWarning)


inner-code-interpreter (to chat_manager):

exitcode: 0 (execution succeeded)
Code output: 
2024-01-02
2024-01-03
2024-01-08
2024-01-09
2024-01-10
2024-01-11
2024-01-12
2024-01-16
2024-01-17
2024-01-18
2024-01-19
2024-01-22
2024-01-23
2024-01-24
2024-01-25
2024-01-26
2024-01-29
2024-01-30
2024-01-31
2024-02-01
2024-02-02
2024-02-05
2024-02-06
2024-02-07
2024-02-08
2024-02-09
2024-02-12
2024-02-13
2024-02-14
2024-02-15
2024-02-16
2024-02-20
2024-02-21
2024-02-22
2024-02-23
2024-02-26
2024-02-27
2024-02-28
2024-02-29
2024-03-01
2024-03-04
2024-03-05
2024-03-06
2024-03-07
2024-03-08
2024-03-11
2024-03-12
2024-03-13
2024-03-14
2024-03-15
2024-03-18
2024-03-19
2024-03-20
2024-03-21
2024-03-22
2024-03-25
2024-03-26
2024-03-27
2024-03-28
2024-04-01
2024-04-02
2024-04-03
2024-04-04
2024-04-05
2024-04-08
2024-04-09
2024-04-10
2024-04-11
2024-04-12
2024-04-15
2024-04-16
2024-04-17
2024-04-18
2024-04-19
2024-04-22
2024-04-23
2024-04-24
2024-04-25
2024-04-26
2024-04-29
2024-04-30
2024-05-01
2024-05

ChatResult(chat_id=None, chat_history=[{'content': 'On which days in 2024 was Microsoft Stock higher than $370?', 'role': 'assistant', 'name': 'user_proxy'}, {'content': "{'content': 'Microsoft Stock was higher than $370 on most trading days from January 2, 2024, to September 24, 2024. The specific days can be categorized by month as follows:\\n\\n- January: 2-31\\n- February: 1-29\\n- March: 1-28\\n- April: 1-30\\n- May: 1-31\\n- June: 3-28\\n- July: 1-31\\n- August: 1-30\\n- September: 3-24', 'refusal': None, 'role': 'assistant', 'function_call': None, 'tool_calls': None}", 'role': 'user', 'name': 'society_of_mind'}], summary="{'content': 'Microsoft Stock was higher than $370 on most trading days from January 2, 2024, to September 24, 2024. The specific days can be categorized by month as follows:\\n\\n- January: 2-31\\n- February: 1-29\\n- March: 1-28\\n- April: 1-30\\n- May: 1-31\\n- June: 3-28\\n- July: 1-31\\n- August: 1-30\\n- September: 3-24', 'refusal': None, 'role': 'assistan

#### Remarks

There are a few things to notice about this output:
- First, the user_proxy sent only one message to the society_of_mind agent, and received only one message in response. As far as it is concerned, the society_of_mind agent is the only agent in the chat.
- Second, the final response is formatted in a way that is standalone. Unlike the prior response, it makes no reference of a previous script or execution, and it lacks the TERMINATE keyword that ended the inner monologue.